In [11]:
## ref : https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/optimize-llama-2-gptq.ipynb 

In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
!sudo -H pip install auto-gptq --no-cache-dir

     |████████████████████████████████| 4.8MB 6.9MB/s eta 0:00:01
     |████████████████████████████████| 266kB 116.4MB/s eta 0:00:01
     |████████████████████████████████| 12.2MB 52.8MB/s eta 0:00:01


  Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [3]:
!sudo -H pip install --upgrade optimum

     |████████████████████████████████| 409kB 7.2MB/s eta 0:00:01


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer, load_quantized_model
import torch

/tmp/pip_packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
model_name = "mistralai/Mistral-7B-v0.1"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16,device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 1 has a total capacty of 15.60 GiB of which 13.00 MiB is free. Process 2680851 has 7.00 GiB memory in use. Process 3703291 has 8.58 GiB memory in use. Of the allocated memory 7.95 GiB is allocated by PyTorch, and 137.70 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
quantizer = GPTQQuantizer(bits=4, dataset="wikitext2")
quantizer.quant_method = "gptq"

In [ ]:
quantized_model = quantizer.quantize_model(model, tokenizer)

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
quant_path = "/data/quantization-trials/GPTQ-quantized/ravi"

In [8]:
# save the quantize model to disk

quantized_model.save_pretrained(quant_path, safe_serialization=True)

### Inference on quantized model

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

In [10]:
gptq_config = GPTQConfig(bits=4, use_exllama=True)

model_id = "/data/quantization-trials/GPTQ-quantized"
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", torch_dtype=torch.float16)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


In [11]:
def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

import time

In [12]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is science")
print("time taken is :", time.time()-start)

time taken is : 133.77150201797485


In [8]:
print(output1)

</s>what is science?

Science is the study of the world around us. It is the study of the physical, chemical, and biological world around us. It is the study of the world around us.

What is the world around us?

The world around us is made up of the physical, chemical, and biological world around us. The world around us is made up of the physical, chemical, and biological world around us.

What is the world around us made of?

The world around us is made up of the physical, chemical, and biological world around us. The world around us is made up of the physical, chemical, and biological world around us.

What is the world around us made of?

The world around us is made up of the physical, chemical, and biological world around us. The world around us is made up of the physical, chemical, and biological world around us.

What is the world around us made of?

The world around us is made up of the physical, chemical, and biological world around us. The world around us is made up of the ph

### Inference on un-quantized model

In [9]:
model_id = "/data/quantization-trials/merged-model"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", torch_dtype=torch.float16)

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [10]:
# Using unquant model
start = time.time()
output2 = predict_from_normal("what is science")
print("time taken is :", time.time()-start)

time taken is : 23.793633222579956


In [11]:
print(output2)

</s>what is science?

The word "science" is used to describe a variety of fields of study, including:

biology

chemistry

biology of the brain

biology of the immune system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the reproductive system

biology of the nervous system

biology of the reproductive system

biology of the immune system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive system

biology of the reproductive sy